# Yelp Businesses: Cleaning and Wrangling

The objective of this notebook is to inspect and wrangle the `business.json` file from the yelp dataset.
At each feature extracting/cleaning step, the data is saved in a separate csv file in the format `business_feature.csv` such that we can trace back the file origin. This is also to avoid ending up with a massing dataframe with too many features.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from collections import Counter, OrderedDict
import calendar
import utils

%matplotlib inline

# Load + Assess

In [2]:
#location of file
business_dir = 'data/business.json'

#download data
df_bus = utils.chunk_loader(business_dir, read_limit=-1)
#df_bus = pd.read_json(business_dir, orient='columns',lines=True)

#head
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [3]:
#shape
df_bus.shape

(192609, 14)

In [4]:
#data types
df_bus.dtypes

address          object
attributes       object
business_id      object
categories       object
city             object
hours            object
is_open           int64
latitude        float64
longitude       float64
name             object
postal_code      object
review_count      int64
stars           float64
state            object
dtype: object

In [5]:
#quick stats
df_bus.describe()

,is_open,latitude,longitude,review_count,stars
count,192609.000000,192609.000000,192609.000000,192609.000000,192609.000000
mean,0.823040,38.541803,-97.594785,33.538962,3.585627
std,0.381635,4.941964,16.697725,110.135224,1.018458
min,0.000000,33.204642,-115.493471,3.000000,1.000000
25%,1.000000,33.637408,-112.274677,4.000000,3.000000
50%,1.000000,36.144815,-111.759323,9.000000,3.500000
75%,1.000000,43.602989,-79.983614,25.000000,4.500000
max,1.000000,51.299943,-72.911982,8348.000000,5.000000


In [6]:
#percent missing
df_bus.isna().mean()

address         0.000000
attributes      0.149713
business_id     0.000000
categories      0.002502
city            0.000000
hours           0.232751
is_open         0.000000
latitude        0.000000
longitude       0.000000
name            0.000000
postal_code     0.000000
review_count    0.000000
stars           0.000000
state           0.000000
dtype: float64

# Cleaning to-do list

Evaluating the business dataframe above, there are several features that need to be cleaned.
The list below offers a roadmap to addressing these issues although it might not be comprehensive. During the process we might need to add additional steps.
We understand that some data types are nested within the columns, and that these data types might not be stored  in the appropirate manner.


- address
    - make everything lower case
    - extract feature: if on road/boulevard/ave/etc...
- attributes
    - break up dict to dummy variables
- business_id
    - no changes
- categories
    - make everything lower?
    - dummy variables and split by comma character
    - note that not everything is a restaurant (plumbers)
- city
    - maybe lower case?
- hours
    - split dict by days
        - open hour monday
        - close hour monday
        - etc...
    - figure out placeholder value for None
    - check if correlation between closed restaurant and no hours posted
- is_open
    - no changes
    - 82% are open, 18% are dead businesses
- latitude
    - no changes
- longitude
    - no changes
- name
    - no changes
- postal_code
    - replace zip code with integer representation
- review_count
    - note that lowest value is 3
- stars
    - no changes
- state
    - some are canadian
    - add feature: is in USA yes/no

In [7]:
#mapping dict for replacing and fixing data types
bool_to_int = {True: 1, False: 0, np.nan: 0, 'True': 1, 'False': 0, 'None': 0, None: 0}

## address

In [8]:
#create deep copy
df_bus_adr = df_bus.copy()
#make everything lower case
df_bus_adr['address'] = df_bus_adr['address'].str.lower()
#remove punctuation
df_bus_adr['address'] = df_bus_adr['address'].str.replace('[^\w\s]','')

In [9]:
#define counter object
adr_counter = Counter()
#loop over every address entry
for add in df_bus_adr.address:
    #loop over each individial word
    for word in add.lower().split():
        #add word to counter
        adr_counter[word] +=1

In [10]:
#list top k words
adr_counter.most_common(100)

[('rd', 46230),
 ('ste', 40472),
 ('w', 29488),
 ('e', 28551),
 ('ave', 23643),
 ('n', 21687),
 ('st', 21172),
 ('s', 20076),
 ('blvd', 18235),
 ('street', 14378),
 ('dr', 12894),
 ('avenue', 9499),
 ('road', 5760),
 ('unit', 5336),
 ('pkwy', 4724),
 ('rue', 4267),
 ('vegas', 3977),
 ('las', 3924),
 ('boulevard', 3185),
 ('100', 3021),
 ('suite', 2921),
 ('yonge', 2907),
 ('sw', 2655),
 ('drive', 2625),
 ('main', 2437),
 ('hwy', 2379),
 ('101', 2378),
 ('1', 2367),
 ('bell', 2293),
 ('queen', 2174),
 ('school', 1971),
 ('scottsdale', 1953),
 ('se', 1890),
 ('center', 1860),
 ('way', 1740),
 ('nw', 1715),
 ('valley', 1657),
 ('ln', 1642),
 ('park', 1585),
 ('110', 1535),
 ('2', 1497),
 ('eastern', 1491),
 ('sahara', 1477),
 ('dundas', 1466),
 ('lake', 1440),
 ('camelback', 1433),
 ('west', 1433),
 ('7', 1431),
 ('university', 1412),
 ('bloor', 1409),
 ('rainbow', 1403),
 ('indian', 1359),
 ('charleston', 1359),
 ('ne', 1351),
 ('105', 1326),
 ('baseline', 1325),
 ('102', 1313),
 ('east'

In [11]:
#road type mapping to homogenize road names
road_type_dict = {'rd': 'road','rue': 'road', 'avenue': 'ave',
                  'street': 'str', 'blvd': 'boulevard',
                  'drive': 'dr', 'highway': 'hwy',
                  'parkway': 'pkwy', 'center': 'ct', 'lane': 'ln'}

#replace names
df_bus_adr['address'] = df_bus_adr['address'].replace(road_type_dict, regex=True)

In [12]:
#get list of finalized road values
#set to remove duplicates
road_types_list = list(set(road_type_dict.values()))
print(road_types_list)

['dr', 'hwy', 'pkwy', 'ln', 'ct', 'road', 'str', 'ave', 'boulevard']


In [13]:
#dict for dummies
road_col_dict = {}
#iterate over road types
for road in road_types_list:
    #create a dummy for that type
    dum_col = df_bus_adr['address'].str.contains(road)
    #add it to the dict
    road_col_dict[road] = dum_col

#convert boolean to 1/0
road_type_df = pd.DataFrame.from_dict(road_col_dict).replace({False:0, True:1})

road_type_df.head()

,dr,hwy,pkwy,ln,ct,road,str,ave,boulevard
0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1


In [14]:
#key for reference in case we want to join tables
road_type_df['business_id'] = df_bus['business_id']

In [15]:
#save the work
road_type_df.to_csv(path_or_buf='data/cleaned/business_roadtype.csv')

### Alternatively we can create a categorical series instead of one hot encoding to reduce the dimensionality of the combined dataframe. We can do this because the categories are mutually exclusive.

In [16]:
#drop index col since we use dataframe later down
road_type_df = road_type_df.drop(columns=['business_id'])

In [72]:
#create dict mapping for road types
road_to_cat_dict = dict(zip(road_type_df.columns, np.arange(1, len(road_type_df.columns)+1)))

#stak dataframe
road_type_df_stack = road_type_df.stack()

#convert to series
road_type_df_cats = pd.Series(pd.Categorical(road_type_df_stack[road_type_df_stack!=0].index.get_level_values(1)))

#replace with dict
road_type_df_cats = road_type_df_cats.replace(road_to_cat_dict)


road_type_df_cats.head()

0    1
1    8
2    6
3    1
4    9
dtype: int64

In [73]:
road_type_df_cats.shape

(161698,)

In [80]:
df_bus['road_type'] = road_type_df_cats

#fill missing values with 0
df_bus['road_type'] = df_bus['road_type'].fillna(0)

### We can't necessarily rank the importance of the address based on its designation. A road might be more visited than a parkway, and the numbering of the road might not indicate anything. Thinking ahead, we believe that using one hot encoding might be the best solution for this categorical variable

### Replace postal codes with corresponding integer code

### Alternatively:
use IRS data to get income by zip code

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi


For Canada:
https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/inc-rev/Table.cfm?Lang=Eng&T=102&PR=0&D1=1&RPP=25&SR=1&S=108&O=D



In [20]:
#get unique postal codes
unique_postal = df_bus['postal_code'].unique().tolist()

#map code to integer representation
postal_to_int = dict(zip(unique_postal, np.arange(0, len(unique_postal))))

#reverse mapping for reference
int_to_postal = {v:k for k,v in postal_to_int.items()}

#replace in dataframe
df_bus['postal_code'] = df_bus['postal_code'].replace(postal_to_int)

## attributes

In [21]:
#break up dict inside df
df_atr = df_bus['attributes'].apply(pd.Series)
df_atr.head(10)

,GoodForKids,RestaurantsReservations,GoodForMeal,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,...,BYOBCorkage,DriveThru,Smoking,AgesAllowed,HairSpecializesIn,Corkage,BYOB,DietaryRestrictions,Open24Hours,RestaurantsCounterService
0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",True,u'loud',True,True,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",False,u'average',True,True,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#inspect types
df_atr.dtypes

GoodForKids                   object
RestaurantsReservations       object
GoodForMeal                   object
BusinessParking               object
Caters                        object
NoiseLevel                    object
RestaurantsTableService       object
RestaurantsTakeOut            object
RestaurantsPriceRange2        object
OutdoorSeating                object
BikeParking                   object
Ambience                      object
HasTV                         object
WiFi                          object
Alcohol                       object
RestaurantsAttire             object
RestaurantsGoodForGroups      object
RestaurantsDelivery           object
BusinessAcceptsCreditCards    object
BusinessAcceptsBitcoin        object
ByAppointmentOnly             object
AcceptsInsurance              object
Music                         object
GoodForDancing                object
CoatCheck                     object
HappyHour                     object
BestNights                    object
W

In [23]:
#select cols that are still as objects
df_atr_obj_cols = df_atr.select_dtypes(include='object').columns
print(df_atr_obj_cols)

Index(['GoodForKids', 'RestaurantsReservations', 'GoodForMeal',
       'BusinessParking', 'Caters', 'NoiseLevel', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'RestaurantsPriceRange2', 'OutdoorSeating',
       'BikeParking', 'Ambience', 'HasTV', 'WiFi', 'Alcohol',
       'RestaurantsAttire', 'RestaurantsGoodForGroups', 'RestaurantsDelivery',
       'BusinessAcceptsCreditCards', 'BusinessAcceptsBitcoin',
       'ByAppointmentOnly', 'AcceptsInsurance', 'Music', 'GoodForDancing',
       'CoatCheck', 'HappyHour', 'BestNights', 'WheelchairAccessible',
       'DogsAllowed', 'BYOBCorkage', 'DriveThru', 'Smoking', 'AgesAllowed',
       'HairSpecializesIn', 'Corkage', 'BYOB', 'DietaryRestrictions',
       'Open24Hours', 'RestaurantsCounterService'],
      dtype='object')


In [24]:
df_atr[df_atr_obj_cols].head(10)

,GoodForKids,RestaurantsReservations,GoodForMeal,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,...,BYOBCorkage,DriveThru,Smoking,AgesAllowed,HairSpecializesIn,Corkage,BYOB,DietaryRestrictions,Open24Hours,RestaurantsCounterService
0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",True,u'loud',True,True,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",False,u'average',True,True,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def str_dict_to_df(series):
    """
    Takes in a pandas series with dicts stored as strings
    returns dataframe with dict keys as columns
    
    serires: pandas series
    """
    eval_list = []
    for sr in series:
        if not pd.isna(sr):
            eval_list.append(eval(sr))
        else:
            eval_list.append(np.nan)
    
    eval_df = pd.Series(eval_list).apply(pd.Series)  
    
    #drop cols that are all nan
    eval_df = eval_df.dropna(axis=1, how='all')
    
    
    return eval_df

In [26]:
#store col names
dict_cols_list = []

for col in df_atr_obj_cols:
    #if contains a curly bracket, then assume column is a dict as string
    if df_atr[col].str.contains('{').any():
        dict_cols_list.append(col)
print(dict_cols_list)            

['GoodForMeal', 'BusinessParking', 'Ambience', 'Music', 'BestNights', 'HairSpecializesIn', 'DietaryRestrictions']


In [27]:
#store dataframes from dict nested columns
dict_col_df_list = []

for col in dict_cols_list:
    #apply string to dict evaluation
    temp_df = str_dict_to_df(df_atr[col])
    #append to list
    dict_col_df_list.append(temp_df)

#combine all in one column
dict_col_df = pd.concat(dict_col_df_list, axis=1)
dict_col_df.head()

/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/range.py:465: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return self._int64index.union(other)
/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/Users/nadimkawwa/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


,dessert,latenight,lunch,dinner,brunch,breakfast,garage,street,validated,lot,...,kids,perms,asian,dairy-free,gluten-free,vegan,kosher,halal,soy-free,vegetarian
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,False,True,True,False,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,True,True,False,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#store dummy attribute columns in list
atr_dum_df_cols = []

#iterate over object columns
for col in df_atr_obj_cols:
    #if contains a categorical variable then it begins with a u
    if df_atr[col].str.contains("u'").any():
        #add to the list
        atr_dum_df_cols.append(col)

#print out list
print(atr_dum_df_cols)

['NoiseLevel', 'WiFi', 'Alcohol', 'RestaurantsAttire', 'BYOBCorkage', 'Smoking', 'AgesAllowed']


In [29]:
#explore unique entries in attribute columns
for col in atr_dum_df_cols:
    print(col)
    print(df_atr[col].unique())
    print(50*'-')


NoiseLevel
[nan "u'loud'" "u'average'" "'average'" "u'quiet'" "'loud'" "'very_loud'"
 "'quiet'" "u'very_loud'" 'None']
--------------------------------------------------
WiFi
[nan "u'no'" "'free'" "u'free'" "'no'" "u'paid'" "'paid'" 'None']
--------------------------------------------------
Alcohol
[nan "u'full_bar'" "u'beer_and_wine'" "u'none'" "'beer_and_wine'" "'none'"
 "'full_bar'" 'None']
--------------------------------------------------
RestaurantsAttire
[nan "u'casual'" "'casual'" "'dressy'" "u'dressy'" "u'formal'" 'None'
 "'formal'"]
--------------------------------------------------
BYOBCorkage
[nan "'no'" "'yes_corkage'" "'yes_free'" "u'no'" "u'yes_free'"
 "u'yes_corkage'" 'None']
--------------------------------------------------
Smoking
[nan "u'no'" "u'outdoor'" "u'yes'" "'no'" 'None' "'outdoor'" "'yes'"]
--------------------------------------------------
AgesAllowed
[nan "u'allages'" "u'21plus'" "u'18plus'" "u'19plus'" 'None']
---------------------------------------------

### We build a dictionary to replace each categorical variable with a numerical value that corresponds to a sliding scale. For example we can rank attire from casual, to dressy, and to formal. This will reduce the feature space, and probably provide more information gain than a one hot encoding. For simplicity we will assume that a NaN is always the lowest value on the scale.

In [30]:
#noise level
noise_level_dict = dict(zip([np.nan, "u'loud'" ,"u'average'" ,
                             "'average'" ,"u'quiet'" ,"'loud'" ,
                             "'very_loud'", "'quiet'" ,"u'very_loud'" ,'None'], 
                            [0, 3, 2, 
                             2, 1, 3, 
                             4, 1 , 4, 0]))
print(noise_level_dict)




{nan: 0, "u'loud'": 3, "u'average'": 2, "'average'": 2, "u'quiet'": 1, "'loud'": 3, "'very_loud'": 4, "'quiet'": 1, "u'very_loud'": 4, 'None': 0}


In [31]:
#free wifi is the best kind of wifi
wifi_dict = dict(zip(
    [np.nan ,"u'no'" ,"'free'" ,
     "u'free'" ,"'no'" ,"u'paid'" ,"'paid'" ,'None'], 
    [0, 1, 3, 
     3, 1, 2, 2, 0]))
print(wifi_dict)

{nan: 0, "u'no'": 1, "'free'": 3, "u'free'": 3, "'no'": 1, "u'paid'": 2, "'paid'": 2, 'None': 0}


In [32]:
#more alcohol is better
alcohol_dict = dict(zip([np.nan ,"u'full_bar'" ,"u'beer_and_wine'" ,
                         "u'none'" ,"'beer_and_wine'" ,"'none'", 
                         "'full_bar'" ,'None'],
                        [0, 2, 1, 
                         0, 1, 0, 
                         2, 0]))
print(alcohol_dict)

{nan: 0, "u'full_bar'": 2, "u'beer_and_wine'": 1, "u'none'": 0, "'beer_and_wine'": 1, "'none'": 0, "'full_bar'": 2, 'None': 0}


In [33]:
#rank by formality
restaurant_attire_dict = dict(zip(
    [np.nan ,"u'casual'" ,"'casual'" ,
     "'dressy'" ,"u'dressy'" ,"u'formal'" ,
     'None',"'formal'"], 
    [0, 1, 1, 
     2, 2, 3, 
     0, 3]))
print(restaurant_attire_dict)

{nan: 0, "u'casual'": 1, "'casual'": 1, "'dressy'": 2, "u'dressy'": 2, "u'formal'": 3, 'None': 0, "'formal'": 3}


In [34]:
#free corkage is best
byob_corkage_dict = dict(zip(
    [np.nan ,"'no'" ,"'yes_corkage'" ,
     "'yes_free'" ,"u'no'" ,"u'yes_free'","u'yes_corkage'"],
    [0, 1, 2, 
     3, 1, 3, 2]))
print(byob_corkage_dict)

{nan: 0, "'no'": 1, "'yes_corkage'": 2, "'yes_free'": 3, "u'no'": 1, "u'yes_free'": 3, "u'yes_corkage'": 2}


In [35]:
#from least to most accomodating for smokers
smoking_dict = dict(zip(
    [np.nan ,"u'no'" ,"u'outdoor'" ,
     "u'yes'" ,"'no'" ,'None' ,
     "'outdoor'" ,"'yes'"], 
    [0, 1, 2, 
     3, 1, 0, 
     2, 3]))
print(smoking_dict)

{nan: 0, "u'no'": 1, "u'outdoor'": 2, "u'yes'": 3, "'no'": 1, 'None': 0, "'outdoor'": 2, "'yes'": 3}


In [36]:
#from youngest to oldest crowd
ages_allowed_dict = dict(zip(
    [np.nan ,"u'allages'" ,"u'21plus'" ,
     "u'18plus'" ,"u'19plus'" ,'None'], 
    [0, 1, 4, 
     2, 3, 0]))
print(ages_allowed_dict)

{nan: 0, "u'allages'": 1, "u'21plus'": 4, "u'18plus'": 2, "u'19plus'": 3, 'None': 0}


In [37]:
#define mapping for replace method
atr_dict_map = {'NoiseLevel': noise_level_dict, 
                'WiFi': wifi_dict, 
                'Alcohol': alcohol_dict, 
                'RestaurantsAttire': restaurant_attire_dict, 
                'BYOBCorkage': byob_corkage_dict, 
                'Smoking': smoking_dict, 
                'AgesAllowed': ages_allowed_dict}

In [38]:
#get a copy with relevant columns
df_atr_categorical = df_atr[atr_dum_df_cols].copy()
#apply replacement
df_atr_categorical = df_atr_categorical.replace(atr_dict_map)

df_atr_categorical.head()

,NoiseLevel,WiFi,Alcohol,RestaurantsAttire,BYOBCorkage,Smoking,AgesAllowed
0,0,0,0,0,0,0,0
1,3,1,2,1,0,0,0
2,2,1,1,1,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


#### The cell block below will return a one hot encoding of the categorical attributes above, uncomment if objective changes.

In [39]:
# #store dummy dataframes in list
# atr_dum_df_list = []

# for col in atr_dum_df_cols:
    
#     #we do not want to modify the dataframe in place so create a copy
#     temp_series = df_atr[col].copy()
    
    
#     #fx messy inputs and remove u
#     temp_series = temp_series.str.replace("u'", "")
#     #remove '
#     temp_series= temp_series.str.replace("'", "")
        
    
#     #create dummies
#     dum_df = pd.get_dummies(temp_series)
#     #drop the None column
#     dum_df = dum_df.drop(columns=['None'])
        
#     #fx messy col names in case we missed them
#     dum_df.columns = dum_df.columns.str.replace("u'", "")
#     dum_df.columns = dum_df.columns.str.replace("'", "")
        
#     #add prefixt
#     dum_df = dum_df.add_prefix(col+'_')
                
#     #append to list
#     atr_dum_df_list.append(dum_df)

# #concat
# atr_dum_df = pd.concat(atr_dum_df_list, axis=1)

# atr_dum_df.head()


In [40]:
# #standard preprocessing for restaurant price range since it has no u
# atr_price_range = pd.get_dummies(df_atr['RestaurantsPriceRange2'])
# atr_price_range = atr_price_range.drop(columns= ['None'])
# atr_price_range = atr_price_range.add_prefix('price_range'+'_')

# atr_price_range.head()

In [41]:
#add to the list
atr_to_drop =  dict_cols_list + list(df_atr_categorical.columns)
print(atr_to_drop)

['GoodForMeal', 'BusinessParking', 'Ambience', 'Music', 'BestNights', 'HairSpecializesIn', 'DietaryRestrictions', 'NoiseLevel', 'WiFi', 'Alcohol', 'RestaurantsAttire', 'BYOBCorkage', 'Smoking', 'AgesAllowed']


In [42]:
#combine features in one dataframes
df_atr_conc = pd.concat([df_atr.drop(columns=atr_to_drop),dict_col_df, df_atr_categorical], axis=1)

#standardize name
df_atr_conc.columns = df_atr_conc.columns.str.replace("-", "_")

#make 1/0
df_atr_conc = df_atr_conc.replace(bool_to_int)

df_atr_conc.head()

,GoodForKids,RestaurantsReservations,Caters,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,BikeParking,HasTV,RestaurantsGoodForGroups,...,halal,soy_free,vegetarian,NoiseLevel,WiFi,Alcohol,RestaurantsAttire,BYOBCorkage,Smoking,AgesAllowed
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,2,0,0,0,1,...,0,0,0,3,1,2,1,0,0,0
2,1,1,0,1,1,2,0,1,1,1,...,0,0,0,2,1,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
#quick check for column names
df_atr_conc.columns

Index(['GoodForKids', 'RestaurantsReservations', 'Caters',
       'RestaurantsTableService', 'RestaurantsTakeOut',
       'RestaurantsPriceRange2', 'OutdoorSeating', 'BikeParking', 'HasTV',
       'RestaurantsGoodForGroups', 'RestaurantsDelivery',
       'BusinessAcceptsCreditCards', 'BusinessAcceptsBitcoin',
       'ByAppointmentOnly', 'AcceptsInsurance', 'GoodForDancing', 'CoatCheck',
       'HappyHour', 'WheelchairAccessible', 'DogsAllowed', 'DriveThru',
       'Corkage', 'BYOB', 'Open24Hours', 'RestaurantsCounterService',
       'dessert', 'latenight', 'lunch', 'dinner', 'brunch', 'breakfast',
       'garage', 'street', 'validated', 'lot', 'valet', 'romantic', 'intimate',
       'classy', 'hipster', 'divey', 'touristy', 'trendy', 'upscale', 'casual',
       'dj', 'background_music', 'no_music', 'jukebox', 'live', 'video',
       'karaoke', 'monday', 'tuesday', 'friday', 'wednesday', 'thursday',
       'sunday', 'saturday', 'straightperms', 'coloring', 'extensions',
       'africana

In [44]:
#key for reference
df_atr_conc['business_id'] = df_bus['business_id']

In [45]:
#save the work
df_atr_conc.to_csv(path_or_buf='data/cleaned/business_attributes.csv')

## Categories

In [46]:
#counter for original categories
cat_counter = Counter()
#loop through split categories
for cat_split in df_bus['categories'].str.split(',| '):
    #if statement to avoid none type is not iterable
    if cat_split:
        for cat in cat_split:
            cat_counter[cat] +=1

In [47]:
#see top k types
cat_counter.most_common(25)

[('', 596232),
 ('&', 129038),
 ('Services', 72809),
 ('Restaurants', 59382),
 ('Food', 47591),
 ('Shopping', 32643),
 ('Home', 31600),
 ('Spas', 23387),
 ('Bars', 21592),
 ('Beauty', 21518),
 ('Medical', 20510),
 ('Health', 18736),
 ('Hair', 15561),
 ('Local', 15405),
 ('Event', 14518),
 ('Repair', 13276),
 ('Automotive', 13203),
 ('Nightlife', 13095),
 ('Stores', 12969),
 ('Salons', 12847),
 ('Planning', 12740),
 ('American', 12580),
 ('Auto', 11392),
 ('Life', 10049),
 ('Arts', 9744)]

In [48]:
#get keys for top k common categories
top_cats = list(dict(cat_counter.most_common(25)).keys())
print(top_cats)
#note that the first 2 were space and & so skip those
print("\nselecting only top 10 relevant sections\n")
top_cats = top_cats[3:13]
print(top_cats)

['', '&', 'Services', 'Restaurants', 'Food', 'Shopping', 'Home', 'Spas', 'Bars', 'Beauty', 'Medical', 'Health', 'Hair', 'Local', 'Event', 'Repair', 'Automotive', 'Nightlife', 'Stores', 'Salons', 'Planning', 'American', 'Auto', 'Life', 'Arts']

selecting only top 10 relevant sections

['Restaurants', 'Food', 'Shopping', 'Home', 'Spas', 'Bars', 'Beauty', 'Medical', 'Health', 'Hair']


In [49]:
#create dict for categories
category_dict = {}
for cat in top_cats:
    #select relevant top 10 categories from before
    dum_cat = df_bus['categories'].str.contains(cat)
    #add it to the dict
    category_dict[cat] = dum_cat
    
cat_type_df = pd.DataFrame.from_dict(category_dict).replace(bool_to_int)

cat_type_df.head()

,Restaurants,Food,Shopping,Home,Spas,Bars,Beauty,Medical,Health,Hair
0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0


In [50]:
#key for reference
cat_type_df['business_id'] = df_bus['business_id']

In [51]:
#save the work
cat_type_df.to_csv(path_or_buf='data/cleaned/business_cats.csv')

## Hours

Holy grail of date time format:

http://strftime.org/

In [52]:
#split out the dict
hours_day_df = df_bus['hours'].apply(pd.Series)
hours_day_df.head()

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9:0-0:0,9:0-0:0,9:0-0:0,9:0-0:0,9:0-1:0,9:0-1:0,9:0-0:0
2,17:30-21:30,NaN,17:30-21:30,17:30-21:30,17:30-22:0,17:30-22:0,17:30-21:0
3,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,NaN,NaN
4,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0


In [53]:
def series_to_datetime(df, series):
    """
    Takes in a pandas series with content stored as a string
    must have format hour:minute - hour:minute
    can have missing values
    
    df = pandas dataframe
    series = pandas column name
    """
    #create array for day of week
    #weekday = list(calendar.day_abbr)
    
    #ordered dict container
    serires_dict = OrderedDict()
    
    #hour container
    open_hour = []
    close_hour = []
    
    #split the series along the dash (-)
    day = df[series].str.split("-")

    #iterate over days
    for hour in day:
        
        #if not a nan the split will return a list
        if type(hour)==list:
            open_hour.append(hour[0])
            close_hour.append(hour[1])
        else:
            #necessary nan for when not available
            open_hour.append(np.nan)
            close_hour.append(np.nan)
            
    #make a datetime object    
    
    open_hour_dt = pd.to_datetime(open_hour, dayfirst=True,format='%H:%M')
    close_hour_dt = pd.to_datetime(close_hour, dayfirst=True, format='%H:%M')
    
    serires_dict[series+'_open'] = open_hour
    serires_dict[series+'_close'] = close_hour
    
    hours_df = pd.DataFrame.from_dict(serires_dict)
    
    return hours_df

In [54]:
hours_df_list = []

for col in hours_day_df.columns:
    temp_hour_df = series_to_datetime(hours_day_df, col)
    hours_df_list.append(temp_hour_df)
    
    
hours_df_openclose = pd.concat(hours_df_list, axis=1)
hours_df_openclose.head()

,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,Thursday_close,Friday_open,Friday_close,Saturday_open,Saturday_close,Sunday_open,Sunday_close
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9:0,0:0,9:0,0:0,9:0,0:0,9:0,0:0,9:0,1:0,9:0,1:0,9:0,0:0
2,17:30,21:30,NaN,NaN,17:30,21:30,17:30,21:30,17:30,22:0,17:30,22:0,17:30,21:0
3,8:0,17:0,8:0,17:0,8:0,17:0,8:0,17:0,8:0,17:0,NaN,NaN,NaN,NaN
4,7:0,23:0,7:0,23:0,7:0,23:0,7:0,23:0,7:0,23:0,7:0,23:0,7:0,23:0


In [55]:
hours_df_openclose.dtypes

Monday_open        object
Monday_close       object
Tuesday_open       object
Tuesday_close      object
Wednesday_open     object
Wednesday_close    object
Thursday_open      object
Thursday_close     object
Friday_open        object
Friday_close       object
Saturday_open      object
Saturday_close     object
Sunday_open        object
Sunday_close       object
dtype: object

In [56]:
#key for reference
hours_df_openclose['business_id'] = df_bus['business_id']

In [57]:
#save the work
hours_df_openclose.to_csv(path_or_buf='data/cleaned/business_hours.csv')

## is_open

In [58]:
#quick check for unique values
df_bus.is_open.unique()

array([0, 1])

## state

In [59]:
#USA! USA!

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [60]:
#get usa sates and make dataframe
df_is_usa = pd.DataFrame(df_bus['state'].str.contains('|'.join(states)).astype(int))
df_is_usa['business_id'] = df_bus['business_id']
#drop redundant column
df_is_usa = df_is_usa.drop(columns=['state'])

In [61]:
#save the work
df_is_usa.to_csv(path_or_buf='data/cleaned/is_usa.csv', header=True)

### Combine DFs in master DF with business_id as common key

In [81]:
# #superseded in favor for utils.reduce_merge

# #combine in massive df
# df_list = [df_bus, 
#            road_type_df_cats, 
#            df_atr_conc.drop(columns=['business_id']), 
#            cat_type_df.drop(columns=['business_id']), 
#            hours_df_openclose.drop(columns=['business_id']), 
#            df_is_usa.drop(columns=['business_id'])]

# df_bus_conc = pd.concat(df_list, axis=1)

# df_bus_conc.shape

In [83]:
#combine in massive df
df_list = [df_bus, 
           df_atr_conc, 
           cat_type_df, 
           hours_df_openclose, 
           df_is_usa]

df_bus_conc = utils.reduce_merge(df_list, key='business_id')

df_bus_conc.shape

(192609, 120)

In [85]:
#check columns
print(*df_bus_conc.columns, sep=',   ')

address,   attributes,   business_id,   categories,   city,   hours,   is_open,   latitude,   longitude,   name,   postal_code,   review_count,   stars,   state,   road_type,   GoodForKids,   RestaurantsReservations,   Caters,   RestaurantsTableService,   RestaurantsTakeOut,   RestaurantsPriceRange2,   OutdoorSeating,   BikeParking,   HasTV,   RestaurantsGoodForGroups,   RestaurantsDelivery,   BusinessAcceptsCreditCards,   BusinessAcceptsBitcoin,   ByAppointmentOnly,   AcceptsInsurance,   GoodForDancing,   CoatCheck,   HappyHour,   WheelchairAccessible,   DogsAllowed,   DriveThru,   Corkage,   BYOB,   Open24Hours,   RestaurantsCounterService,   dessert,   latenight,   lunch,   dinner,   brunch,   breakfast,   garage,   street,   validated,   lot,   valet,   romantic,   intimate,   classy,   hipster,   divey,   touristy,   trendy,   upscale,   casual,   dj,   background_music,   no_music,   jukebox,   live,   video,   karaoke,   monday,   tuesday,   friday,   wednesday,   thursday,   su

In [88]:
#save the work
df_bus_conc.to_csv(path_or_buf='data/cleaned/df_bus_conc.csv', header=True)
